# MAGYC Example - Simulated Data Calibration & Self Evaluation

This notebook evaluates the MAGYC and benchmark algorithms formagnetometer and gyroscope calibration using simulated data. The calibration dataset corresponds to the simulated data used in the paper: "Full Magnetometer and Gyroscope Bias Estimation Using Angular Rates: Theory and Experimental Evaluation of a Factor Graph-Based Approach" by S. Rodríguez-Martínez and G. Troni, 2024. The dataset is available on Google Drive.

> For this notebook to work, it is required to use `example/20241107_sim_calibration.ipynb` to get the results for the calibration in the WAM, MAM and LAM datasets. These results are saved in the `example` folder.

## Set dependencies

In [ ]:
import os
import pickle as pkl
import sys

import gdown
import matplotlib.pyplot as plt
import navlib.math as nm
import navlib.nav as nn
import numpy as np
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, "magyc")))

import magyc

## Import Data from Google Drive

In [ ]:
# Paths
cwd_path = os.path.abspath(os.getcwd())
sim_data_path = os.path.join(os.path.abspath(os.getcwd()), "sim_magcal_20241107_2049.pkl")

if not os.path.isfile(sim_data_path):
    # Google Drive file id
    file_id = "1c5Y1y3PU0pYVrRuZwQGtYYlmj3X2twRm"
    url = f"https://drive.google.com/uc?id={file_id}"

    # Download the file
    gdown.download(url, sim_data_path)

## Load Data

In [ ]:
# Read pickle
with open(sim_data_path, 'rb') as f:
    data = pkl.load(f)

# For Evaluation, we use WAM, as it is the dataset that covers the largest area of the magnetic field ellipsoid
data = data["high"]

In [ ]:
# Extract data
time = data["t"]
magnetic_field = data["mmt"]
angular_rates = data["wmt"]
rph = data["rph"]
si_gt = data["si"][0]
hi_gt = data["hi"][0]
wb_gt = data["wb"][0]

## Load Calibration

In [ ]:
# Load calibration files.
results_SI_wam = pkl.load(open(f"{cwd_path}/calibration_si_wam.pkl", "rb"))
results_HI_wam = pkl.load(open(f"{cwd_path}/calibration_hi_wam.pkl", "rb"))
results_WB_wam = pkl.load(open(f"{cwd_path}/calibration_wb_wam.pkl", "rb"))

results_SI_mam = pkl.load(open(f"{cwd_path}/calibration_si_mam.pkl", "rb"))
results_HI_mam = pkl.load(open(f"{cwd_path}/calibration_hi_mam.pkl", "rb"))
results_WB_mam = pkl.load(open(f"{cwd_path}/calibration_wb_mam.pkl", "rb"))

results_SI_lam = pkl.load(open(f"{cwd_path}/calibration_si_lam.pkl", "rb"))
results_HI_lam = pkl.load(open(f"{cwd_path}/calibration_hi_lam.pkl", "rb"))
results_WB_lam = pkl.load(open(f"{cwd_path}/calibration_wb_lam.pkl", "rb"))

# Add Raw Value
results_SI_wam["RAW"] = np.eye(3)
results_HI_wam["RAW"] = np.zeros((3, 1))
results_WB_wam["RAW"] = np.zeros((3, 1))

results_SI_mam["RAW"] = np.eye(3)
results_HI_mam["RAW"] = np.zeros((3, 1))
results_WB_mam["RAW"] = np.zeros((3, 1))

results_SI_lam["RAW"] = np.eye(3)
results_HI_lam["RAW"] = np.zeros((3, 1))
results_WB_lam["RAW"] = np.zeros((3, 1))

## Compute corrected Magnetic Field, heading and error metrics

In [ ]:
magc_cross_wam, rph_est_cross_wam, magc_std_cross_wam, rph_rmse_cross_wam = {}, {}, {}, {}
magc_cross_mam, rph_est_cross_mam, magc_std_cross_mam, rph_rmse_cross_mam = {}, {}, {}, {}
magc_cross_lam, rph_est_cross_lam, magc_std_cross_lam, rph_rmse_cross_lam = {}, {}, {}, {}
magc_cross_tam, rph_est_cross_tam, magc_std_cross_tam, rph_rmse_cross_tam = {}, {}, {}, {}

print("--- WAM ---")
for k in results_HI_wam.keys():
    print(f"Correcting magnetic field and RPH for: {k}")
    HI, SI, WB = results_HI_wam[k], results_SI_wam[k], results_WB_wam[k]

    # Check if the calibration is valid
    if HI is None or np.any(np.isnan(HI)) or np.any(np.isnan(SI)):
        magc_cross_wam[k] = None
        magc_std_cross_wam[k] = None
        rph_est_cross_wam[k] = None
        rph_rmse_cross_wam[k] = None
        continue

    # Correct the magnetic field and compute magnetic field std
    correct_magfield = np.copy(magnetic_field)
    std_accumulator = [0, 0]
    for i in range(correct_magfield.shape[0]):
        correct_magfield[i] = (np.linalg.inv(SI) @ (magnetic_field[i] - HI.flatten()).T).T
        std_accumulator[0] += nm.std(nm.norm(correct_magfield[i])) * 1e3
        std_accumulator[1] += 1
    magc_std_cross_wam[k] = std_accumulator[0] / std_accumulator[1]
    magc_cross_wam[k] = np.copy(correct_magfield)

    # Compute the heading and the heading rmse
    rph_est = np.copy(rph)
    hdg_rmse = [0, 0]
    for i in range(rph_est.shape[0]):
        rph_est[i, :, -1] = nn.ahrs_raw_hdg(correct_magfield[i], np.concatenate([rph_est[i, :, :-1], np.zeros((rph_est.shape[1], 1))], axis=1)).squeeze()
        hdg_rmse[0] += np.sqrt(nm.mean(nm.wrapunwrap(rph_est[i, :, -1] - rph[i, :, -1]) ** 2))
        hdg_rmse[1] += 1
    rph_est_cross_wam[k] = np.copy(rph_est)
    rph_rmse_cross_wam[k] = np.rad2deg(hdg_rmse[0] / hdg_rmse[1])

print("\n--- MAM ---")
for k in results_HI_mam.keys():
    print(f"Correcting magnetic field and RPH for: {k}")
    HI, SI, WB = results_HI_mam[k], results_SI_mam[k], results_WB_mam[k]

    # Check if the calibration is valid
    if HI is None or np.any(np.isnan(HI)) or np.any(np.isnan(SI)):
        magc_cross_mam[k] = None
        magc_std_cross_mam[k] = None
        rph_est_cross_mam[k] = None
        rph_rmse_cross_mam[k] = None
        continue

    # Correct the magnetic field and compute magnetic field std
    correct_magfield = np.copy(magnetic_field)
    std_accumulator = [0, 0]
    for i in range(correct_magfield.shape[0]):
        correct_magfield[i] = (np.linalg.inv(SI) @ (magnetic_field[i] - HI.flatten()).T).T
        std_accumulator[0] += nm.std(nm.norm(correct_magfield[i])) * 1e3
        std_accumulator[1] += 1
    magc_std_cross_mam[k] = std_accumulator[0] / std_accumulator[1]
    magc_cross_mam[k] = np.copy(correct_magfield)

    # Compute the heading and the heading rmse
    rph_est = np.copy(rph)
    hdg_rmse = [0, 0]
    for i in range(rph_est.shape[0]):
        rph_est[i, :, -1] = nn.ahrs_raw_hdg(correct_magfield[i], np.concatenate([rph_est[i, :, :-1], np.zeros((rph_est.shape[1], 1))], axis=1)).squeeze()
        hdg_rmse[0] += np.sqrt(nm.mean(nm.wrapunwrap(rph_est[i, :, -1] - rph[i, :, -1]) ** 2))
        hdg_rmse[1] += 1
    rph_est_cross_mam[k] = np.copy(rph_est)
    rph_rmse_cross_mam[k] = np.rad2deg(hdg_rmse[0] / hdg_rmse[1])

print("\n--- LAM ---")
for k in results_HI_lam.keys():
    print(f"Correcting magnetic field and RPH for: {k}")
    HI, SI, WB = results_HI_lam[k], results_SI_lam[k], results_WB_lam[k]

    # Check if the calibration is valid
    if HI is None or np.any(np.isnan(HI)) or np.any(np.isnan(SI)):
        magc_cross_lam[k] = None
        magc_std_cross_lam[k] = None
        rph_est_cross_lam[k] = None
        rph_rmse_cross_lam[k] = None
        continue

    # Correct the magnetic field and compute magnetic field std
    correct_magfield = np.copy(magnetic_field)
    std_accumulator = [0, 0]
    for i in range(correct_magfield.shape[0]):
        correct_magfield[i] = (np.linalg.inv(SI) @ (magnetic_field[i] - HI.flatten()).T).T
        std_accumulator[0] += nm.std(nm.norm(correct_magfield[i])) * 1e3
        std_accumulator[1] += 1
    magc_std_cross_lam[k] = std_accumulator[0] / std_accumulator[1]
    magc_cross_lam[k] = np.copy(correct_magfield)

    # Compute the heading and the heading rmse
    rph_est = np.copy(rph)
    hdg_rmse = [0, 0]
    for i in range(rph_est.shape[0]):
        rph_est[i, :, -1] = nn.ahrs_raw_hdg(correct_magfield[i], np.concatenate([rph_est[i, :, :-1], np.zeros((rph_est.shape[1], 1))], axis=1)).squeeze()
        hdg_rmse[0] += np.sqrt(nm.mean(nm.wrapunwrap(rph_est[i, :, -1] - rph[i, :, -1]) ** 2))
        hdg_rmse[1] += 1
    rph_est_cross_lam[k] = np.copy(rph_est)
    rph_rmse_cross_lam[k] = np.rad2deg(hdg_rmse[0] / hdg_rmse[1])

## Soft-Iron Error

To compute the soft-iron error, we leverage the geodesic distance between two positive definite symmetrics matrices based on Bhatia (2007) [1]. This metrics is the affine-invariant Riemannian distance between two positive definite symmetric matrices.

[1] Bhatia, R. (2007). Positive Definite Matrices. Princeton: Princeton University Press. https://doi.org/10.1515/9781400827787


In [ ]:
soft_iron_cross_wam = {}
soft_iron_cross_mam = {}
soft_iron_cross_lam = {}

for k in results_SI_wam.keys():
    if results_SI_wam[k] is None or np.any(np.isnan(results_SI_wam[k])):
        soft_iron_cross_wam[k] = None
        continue
    # Compute error
    soft_iron_cross_wam[k] = magyc.pds_geodesic_distance(results_SI_wam[k], si_gt)

for k in results_SI_mam.keys():
    if results_SI_mam[k] is None or np.any(np.isnan(results_SI_mam[k])):
        soft_iron_cross_mam[k] = None
        continue
    # Compute error
    soft_iron_cross_mam[k] = magyc.pds_geodesic_distance(results_SI_mam[k], si_gt)

for k in results_SI_lam.keys():
    if results_SI_lam[k] is None or np.any(np.isnan(results_SI_lam[k])):
        soft_iron_cross_lam[k] = None
        continue
    # Compute error
    soft_iron_cross_lam[k] = magyc.pds_geodesic_distance(results_SI_lam[k], si_gt)

# Merge all into a single dictionary
soft_iron_cross = {}
for si_wam, si_mam, si_lam in zip(soft_iron_cross_wam.items(),
                                  soft_iron_cross_mam.items(),
                                  soft_iron_cross_lam.items()):
    k = si_wam[0]
    soft_iron_cross[k] = (si_wam[1], si_mam[1], si_lam[1])

## Hard Iron Error

Computed as a simple euclidean distance.

In [ ]:
hard_iron_cross_wam = {}
hard_iron_cross_mam = {}
hard_iron_cross_lam = {}

for k in results_HI_wam.keys():
    if results_HI_wam[k] is None:
        hard_iron_cross_wam[k] = None
        continue
    # Compute error
    hard_iron_cross_wam[k] = np.linalg.norm(results_HI_wam[k].flatten() - hi_gt.flatten()) * 1e3

for k in results_HI_mam.keys():
    if results_HI_mam[k] is None:
        hard_iron_cross_mam[k] = None
        continue
    # Compute error
    hard_iron_cross_mam[k] = np.linalg.norm(results_HI_mam[k].flatten() - hi_gt.flatten()) * 1e3

for k in results_HI_lam.keys():
    if results_HI_lam[k] is None:
        hard_iron_cross_lam[k] = None
        continue
    # Compute error
    hard_iron_cross_lam[k] = np.linalg.norm(results_HI_lam[k].flatten() - hi_gt.flatten()) * 1e3

# Merge all into a single dictionary
hard_iron_cross = {}

# Merge all into a single dictionary
hard_iron_cross = {}
for hi_wam, hi_mam, hi_lam in zip(hard_iron_cross_wam.items(),
                                  hard_iron_cross_mam.items(),
                                  hard_iron_cross_lam.items()):
    k = hi_wam[0]
    hard_iron_cross[k] = (hi_wam[1], hi_mam[1], hi_lam[1])

## Gyroscope bias error

Computed as a simple euclidean distance

In [ ]:
gyro_bias_cross_wam = {}
gyro_bias_cross_mam = {}
gyro_bias_cross_lam = {}

for k in results_WB_wam.keys():
    if results_WB_wam[k] is None:
        gyro_bias_cross_wam[k] = None
        continue
    # Compute error
    gyro_bias_cross_wam[k] = np.linalg.norm(results_WB_wam[k].flatten() - wb_gt.flatten()) * 1e3

for k in results_WB_mam.keys():
    if results_WB_mam[k] is None:
        gyro_bias_cross_mam[k] = None
        continue
    # Compute error
    gyro_bias_cross_mam[k] = np.linalg.norm(results_WB_mam[k].flatten() - wb_gt.flatten()) * 1e3

for k in results_WB_lam.keys():
    if results_WB_lam[k] is None:
        gyro_bias_cross_lam[k] = None
        continue
    # Compute error
    gyro_bias_cross_lam[k] = np.linalg.norm(results_WB_lam[k].flatten() - wb_gt.flatten()) * 1e3

# Merge all into a single dictionary
gyro_bias_cross = {}
for wb_wam, wb_mam, wb_lam in zip(gyro_bias_cross_wam.items(),
                                  gyro_bias_cross_mam.items(),
                                  gyro_bias_cross_lam.items()):
    k = wb_wam[0]
    gyro_bias_cross[k] = (wb_wam[1], wb_mam[1], wb_lam[1])

### Generate table with results

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

data_matrix_errors_si = {}
data_matrix_errors_si["Methods"] = ["SI GD WAM", "SI GD MAM", "SI GD LAM"]
data_matrix_errors_si["RAW"] = [i for i in soft_iron_cross["RAW"]]
data_matrix_errors_si["MAGYC-BFG"] = [i for i in soft_iron_cross["magyc_bfg"]]
data_matrix_errors_si["Ellipsoid Fit"] = [i for i in soft_iron_cross["ellipsoid_fit"]]
data_matrix_errors_si["TWOSTEP"] = [i for i in soft_iron_cross["twostep"]]
data_matrix_errors_si["MagFactor3"] = [i for i in soft_iron_cross["magfactor3"]]
data_matrix_errors_si["MAGYC-IFG"] = [i for i in soft_iron_cross["magyc_ifg"]]

df_si = pd.DataFrame(data_matrix_errors_si)
print("RESULTS FOR SOFT IRON RIEMMANIAN GEODESIC DISTANCE\n")
print(df_si)

data_matrix_errors_hi = {}
data_matrix_errors_hi["Methods"] = ["HI L2 WAM (mG)", "HI L2 MAM (mG)", "HI L2 LAM (mG)"]
data_matrix_errors_hi["RAW"] = [i for i in hard_iron_cross["RAW"]]
data_matrix_errors_hi["MAGYC-BFG"] = [i for i in hard_iron_cross["magyc_bfg"]]
data_matrix_errors_hi["Ellipsoid Fit"] = [i for i in hard_iron_cross["ellipsoid_fit"]]
data_matrix_errors_hi["TWOSTEP"] = [i for i in hard_iron_cross["twostep"]]
data_matrix_errors_hi["MagFactor3"] = [i for i in hard_iron_cross["magfactor3"]]
data_matrix_errors_hi["MAGYC-IFG"] = [i for i in hard_iron_cross["magyc_ifg"]]
df_hi = pd.DataFrame(data_matrix_errors_hi)
print("\nRESULTS FOR HARD IRON L2 NORM\n")
print(df_hi)

data_matrix_errors_wb = {}
data_matrix_errors_wb["Methods"] = ["WB L2 WAM (mrad/s)", "WB L2 MAM (mrad/s)", "WB L2 LAM (mrad/s)"]
data_matrix_errors_wb["RAW"] = [i for i in gyro_bias_cross["RAW"]]
data_matrix_errors_wb["MAGYC-BFG"] = [i for i in gyro_bias_cross["magyc_bfg"]]
data_matrix_errors_wb["Ellipsoid Fit"] = [i for i in gyro_bias_cross["ellipsoid_fit"]]
data_matrix_errors_wb["TWOSTEP"] = [i for i in gyro_bias_cross["twostep"]]
data_matrix_errors_wb["MagFactor3"] = [i for i in gyro_bias_cross["magfactor3"]]
data_matrix_errors_wb["MAGYC-IFG"] = [i for i in gyro_bias_cross["magyc_ifg"]]
df_wb = pd.DataFrame(data_matrix_errors_wb)
print("\nRESULTS FOR GYRO BIAS L2 NORM\n")
print(df_wb)

## Evaluation Metrics

As evaluation metrics, we will compute the mean heading RMSE and the corrected magnetic field standard deviation

In [ ]:
# Heading rmse
hdg_rmse = {}
hdg_rmse["Methods"] = ["Mean Heading RMSE WAM (deg)", "Mean Heading RMSE MAM (deg)", "Mean Heading RMSE LAM (deg)"]
hdg_rmse["RAW"] = [rph_rmse_cross_wam["RAW"], rph_rmse_cross_mam["RAW"], rph_rmse_cross_lam["RAW"]]
hdg_rmse["MAGYC-BFG"] = [rph_rmse_cross_wam["magyc_bfg"], rph_rmse_cross_mam["magyc_bfg"], rph_rmse_cross_lam["magyc_bfg"]]
hdg_rmse["Ellipsoid Fit"] = [rph_rmse_cross_wam["ellipsoid_fit"], rph_rmse_cross_mam["ellipsoid_fit"], rph_rmse_cross_lam["ellipsoid_fit"]]
hdg_rmse["TWOSTEP"] = [rph_rmse_cross_wam["twostep"], rph_rmse_cross_mam["twostep"], rph_rmse_cross_lam["twostep"]]
hdg_rmse["MagFactor3"] = [rph_rmse_cross_wam["magfactor3"], rph_rmse_cross_mam["magfactor3"], rph_rmse_cross_lam["magfactor3"]]
hdg_rmse["MAGYC-IFG"] = [rph_rmse_cross_wam["magyc_ifg"], rph_rmse_cross_mam["magyc_ifg"], rph_rmse_cross_lam["magyc_ifg"]]

df_hdg_rmse = pd.DataFrame(hdg_rmse)
print("\nRESULTS FOR MEAN HEADING RMSE\n")
print(df_hdg_rmse)

# Magnetic field standard deviation
mag_std = {}
mag_std["Methods"] = ["Mean Magnetic Field Std WAM (mG)", "Mean Magnetic Field Std MAM (mG)", "Mean Magnetic Field Std LAM (mG)"]
mag_std["RAW"] = [magc_std_cross_wam["RAW"], magc_std_cross_mam["RAW"], magc_std_cross_lam["RAW"]]
mag_std["MAGYC-BFG"] = [magc_std_cross_wam["magyc_bfg"], magc_std_cross_mam["magyc_bfg"], magc_std_cross_lam["magyc_bfg"]]
mag_std["Ellipsoid Fit"] = [magc_std_cross_wam["ellipsoid_fit"], magc_std_cross_mam["ellipsoid_fit"], magc_std_cross_lam["ellipsoid_fit"]]
mag_std["TWOSTEP"] = [magc_std_cross_wam["twostep"], magc_std_cross_mam["twostep"], magc_std_cross_lam["twostep"]]
mag_std["MagFactor3"] = [magc_std_cross_wam["magfactor3"], magc_std_cross_mam["magfactor3"], magc_std_cross_lam["magfactor3"]]
mag_std["MAGYC-IFG"] = [magc_std_cross_wam["magyc_ifg"], magc_std_cross_mam["magyc_ifg"], magc_std_cross_lam["magyc_ifg"]]

df_mag_std = pd.DataFrame(mag_std)
print("\nRESULTS FOR MEAN MAGNETIC FIELD STD\n")
print(df_mag_std)